In [1]:
!pip install yfinance pandas pyyaml nsepy yahoofinancials matplotlib seaborn scipy sqlalchemy mysql-connector-python

In [2]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Step 2: Load the combined CSV file
# Assuming you've run the previous scripts and have the combined CSV file
combined_csv_path = "nifty50_csv_files/nifty50_full_year_2024.csv"

# Load the data
df = pd.read_csv(combined_csv_path)

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

Dataset Shape: (12300, 8)

First 5 rows:


,Stock,Date,Open,High,Low,Close,Volume,Sector
0,ADANIPORTS.NS,2024-01-01,1017.22,1048.33,1013.70,1038.32,3989711,Industrials
1,ADANIPORTS.NS,2024-01-02,1038.32,1073.05,1021.72,1068.59,6344621,Industrials
2,ADANIPORTS.NS,2024-01-03,1094.95,1133.60,1053.09,1084.30,33060778,Industrials
3,ADANIPORTS.NS,2024-01-04,1106.84,1120.72,1093.52,1112.99,9771995,Industrials
4,ADANIPORTS.NS,2024-01-05,1118.83,1149.45,1115.17,1143.75,10622789,Industrials


In [4]:
# Step 3: Check data types and convert if necessary
print("Data types before conversion:")
print(df.dtypes)

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Ensure numeric columns are properly formatted
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nData types after conversion:")
print(df.dtypes)

Data types before conversion:
Stock      object
Date       object
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
Sector     object
dtype: object

Data types after conversion:
Stock             object
Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
Sector            object
dtype: object


In [5]:
# Step 4: Check for missing values
print("Missing values in each column:")
missing_values = df.isnull().sum()
print(missing_values)



Missing values in each column:
Stock     0
Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Sector    0
dtype: int64


In [6]:
# Step 5: Handle missing values
# For OHLC data, we can forward fill then backward fill
df_clean = df.copy()

# Group by stock and fill missing values within each stock
df_clean = df_clean.groupby('Stock').apply(
    lambda group: group.ffill().bfill()
).reset_index(drop=True)

# Check if any missing values remain
print("Missing values after cleaning:")
print(df_clean.isnull().sum())

Missing values after cleaning:
Stock     0
Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Sector    0
dtype: int64


C:\Users\Balamurugan\AppData\Local\Temp\ipykernel_20088\3987364102.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_clean = df_clean.groupby('Stock').apply(


In [7]:
# Step 6: Check for duplicates
print("Number of duplicate rows:", df_clean.duplicated().sum())

# Remove duplicates if any
df_clean = df_clean.drop_duplicates()

print("Number of duplicate rows after cleaning:", df_clean.duplicated().sum())

Number of duplicate rows: 0
Number of duplicate rows after cleaning: 0


In [8]:
# Step 7: Validate data consistency
# Check for negative prices or volumes
print("Rows with negative Open prices:", (df_clean['Open'] <= 0).sum())
print("Rows with negative High prices:", (df_clean['High'] <= 0).sum())
print("Rows with negative Low prices:", (df_clean['Low'] <= 0).sum())
print("Rows with negative Close prices:", (df_clean['Close'] <= 0).sum())
print("Rows with negative Volume:", (df_clean['Volume'] < 0).sum())

# Check if High is greater than or equal to Low
inconsistent_high_low = (df_clean['High'] < df_clean['Low']).sum()
print("Rows where High < Low:", inconsistent_high_low)

# Check if Open and Close are between High and Low
inconsistent_ohlc = ((df_clean['Open'] > df_clean['High']) | 
                     (df_clean['Open'] < df_clean['Low']) |
                     (df_clean['Close'] > df_clean['High']) | 
                     (df_clean['Close'] < df_clean['Low'])).sum()
print("Rows with inconsistent OHLC values:", inconsistent_ohlc)

Rows with negative Open prices: 0
Rows with negative High prices: 0
Rows with negative Low prices: 0
Rows with negative Close prices: 0
Rows with negative Volume: 0
Rows where High < Low: 0
Rows with inconsistent OHLC values: 0


In [9]:
import pandas as pd

# Example: df_clean already loaded
# df_clean = pd.read_csv("nifty50_full_year_2024.csv")

# -----------------------------
# Step 1: Strip column names to avoid extra spaces
# -----------------------------
df_clean.columns = df_clean.columns.str.strip()

# -----------------------------
# Step 2: Ensure 'Date' is datetime
# -----------------------------
if 'Date' in df_clean.columns:
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
else:
    raise KeyError("No 'Date' column found in df_clean!")

# -----------------------------
# Step 3: Sort by Stock and Date
# -----------------------------
df_clean = df_clean.sort_values(['Stock', 'Date'])

# -----------------------------
# Step 4: Calculate Daily Return (to demonstrate how to remove it)
# -----------------------------
df_clean['Daily_Return'] = df_clean.groupby('Stock')['Close'].pct_change()

# -----------------------------
# Step 5: Remove the Daily_Return column
# -----------------------------
if 'Daily_Return' in df_clean.columns:
    df_clean = df_clean.drop('Daily_Return', axis=1)

# -----------------------------
# Step 6: Check final DataFrame
# -----------------------------
print(df_clean.head())
print(df_clean.columns.tolist())

         Stock       Date     Open     High      Low    Close    Volume  \
0  ADANIENT.NS 2024-01-01  2849.69  2944.31  2839.45  2914.53   2898619   
1  ADANIENT.NS 2024-01-02  2919.33  2945.51  2838.40  2929.72   2671368   
2  ADANIENT.NS 2024-01-03  3044.22  3196.08  2952.90  3000.26  19725411   
3  ADANIENT.NS 2024-01-04  3036.62  3041.67  2987.27  2995.56   2975620   
4  ADANIENT.NS 2024-01-05  3007.20  3064.10  2978.38  3003.85   3219949   

   Sector  
0  Energy  
1  Energy  
2  Energy  
3  Energy  
4  Energy  
['Stock', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Sector']


In [10]:
# Step 13: Save the cleaned data
# Create a new directory for cleaned data
os.makedirs('nifty50_cleaned_data', exist_ok=True)

# Save the cleaned dataframe
cleaned_csv_path = r"D:\python_programs\NIFTY_50 STOCKS_ANALYSIS\nifty50_cleaned_data\nifty50_cleaned_2024.csv"
df_clean.to_csv(cleaned_csv_path, index=False)

print(f"Cleaned data saved to: {cleaned_csv_path}")
print(f"Final dataset shape: {df_clean.shape}")

Cleaned data saved to: D:\python_programs\NIFTY_50 STOCKS_ANALYSIS\nifty50_cleaned_data\nifty50_cleaned_2024.csv
Final dataset shape: (12300, 8)


In [11]:
# Import MySQL libraries
import sqlalchemy
from sqlalchemy import create_engine, text
import mysql.connector


In [12]:


db_config = {
    "host": "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    "port": 4000,
    "user": "2JRRhPHCS6mRsGW.root",
    "password": "48euDwRpY6OmYT5A",
    "ssl_ca": r"D:\path\to\ca.pem"  # replace with your CA cert path
}

# Connect without specifying database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create new database
cursor.execute("CREATE DATABASE IF NOT EXISTS nifty50_project;")

print("✅ Database nifty50_project created successfully!")

cursor.close()
conn.close()


✅ Database nifty50_project created successfully!


In [13]:
from sqlalchemy import create_engine

# --- Use your existing db_config ---
db_config = {
    "host": "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    "port": 4000,
    "user": "2JRRhPHCS6mRsGW.root",
    "password": "48euDwRpY6OmYT5A",
    "database": "nifty50_project",
    "ssl_ca": r"D:\path\to\ca.pem"
}

# --- Build connection string ---
conn_str = (
    f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    f"?ssl_ca={db_config['ssl_ca']}"
)

engine = create_engine(conn_str)

# --- Save cleaned DataFrame into MySQL ---
df_clean.to_sql(
    name="nifty50_data",
    con=engine,
    if_exists="replace",   # replaces old table with new schema
    index=False
)



print("✅ df_clean saved into MySQL table 'nifty50_data' successfully!")


✅ df_clean saved into MySQL table 'nifty50_data' successfully!


In [14]:
row_count = pd.read_sql("SELECT COUNT(*) AS total FROM nifty50_data", con=engine)
print("Total rows:", row_count["total"][0])


Total rows: 12300


In [15]:
df_symbols = pd.read_sql("SELECT DISTINCT stock FROM nifty50_data", con=engine)
print(df_symbols)


            stock
0   TATACONSUM.NS
1     AXISBANK.NS
2       MARUTI.NS
3        CIPLA.NS
4    EICHERMOT.NS
5     HDFCLIFE.NS
6   BHARTIARTL.NS
7        TITAN.NS
8         NTPC.NS
9      SBILIFE.NS
10  BAJFINANCE.NS
11  ULTRACEMCO.NS
12        SBIN.NS
13    ADANIENT.NS
14         UPL.NS
15   POWERGRID.NS
16         BEL.NS
17       TECHM.NS
18  ADANIPORTS.NS
19     HCLTECH.NS
20  HEROMOTOCO.NS
21        INFY.NS
22    DIVISLAB.NS
23    HINDALCO.NS
24  APOLLOHOSP.NS
25         ITC.NS
26  BAJAJ-AUTO.NS
27   SUNPHARMA.NS
28        LTIM.NS
29   BRITANNIA.NS
30         M&M.NS
31    HDFCBANK.NS
32         TCS.NS
33    RELIANCE.NS
34   TATASTEEL.NS
35   NESTLEIND.NS
36    JSWSTEEL.NS
37   ICICIBANK.NS
38      GRASIM.NS
39        ZEEL.NS
40          LT.NS
41   COALINDIA.NS
42  INDUSINDBK.NS
43     DRREDDY.NS
44  ASIANPAINT.NS
45       WIPRO.NS
46        ONGC.NS
47  TATAMOTORS.NS
48   KOTAKBANK.NS
49  BAJAJFINSV.NS


In [16]:
print(df_clean)

             Stock       Date     Open     High      Low    Close    Volume  \
0      ADANIENT.NS 2024-01-01  2849.69  2944.31  2839.45  2914.53   2898619   
1      ADANIENT.NS 2024-01-02  2919.33  2945.51  2838.40  2929.72   2671368   
2      ADANIENT.NS 2024-01-03  3044.22  3196.08  2952.90  3000.26  19725411   
3      ADANIENT.NS 2024-01-04  3036.62  3041.67  2987.27  2995.56   2975620   
4      ADANIENT.NS 2024-01-05  3007.20  3064.10  2978.38  3003.85   3219949   
...            ...        ...      ...      ...      ...      ...       ...   
12295      ZEEL.NS 2024-12-24   123.17   124.93   123.14   123.55   4459275   
12296      ZEEL.NS 2024-12-26   123.81   124.26   121.84   122.28   3910967   
12297      ZEEL.NS 2024-12-27   121.95   123.09   121.55   122.15   2498981   
12298      ZEEL.NS 2024-12-30   121.42   122.25   119.11   119.59   4421171   
12299      ZEEL.NS 2024-12-31   118.71   119.86   117.55   118.80   4445600   

                       Sector  
0                  